# 🚀 **Módulo 1: Introdução ao LangChain - O Quebra-Cabeça da IA**

## **Aula 1.1: O que é LangChain e por que todo mundo tá falando disso?**

---

### **Tá, mas o que é LangChain mesmo?**

Imagina que você é um pedreiro e tem um monte de tijolos, cimento, areia e ferramentas espalhadas pelo terreno. Você **pode** construir uma casa, mas vai ser um trabalho do caralho e vai demorar uma eternidade.

Agora imagina que alguém te dá um **kit de construção** com tudo organizado, instruções claras e até uns moldes prontos. Muito mais fácil, né?

**LangChain é exatamente isso para IA!** 🧱

Sem LangChain, você tem que:
- Conectar APIs manualmente
- Gerenciar memória de conversas
- Implementar prompts do zero
- Fazer parsing de respostas
- E mais um monte de coisa chata

Com LangChain, você tem **componentes prontos** que se encaixam como peças de Lego. É tipo ter um **"Lego da IA"** - você junta as peças e faz coisas incríveis sem reinventar a roda.

### **Por que LangChain é tipo um "pedreiro inteligente"?**

LangChain não é só uma biblioteca, é um **framework completo** que te ajuda a:

1. **Organizar prompts** (como ter receitas de bolo prontas)
2. **Conectar diferentes IAs** (como ter um tradutor que fala com um analista)
3. **Lembrar de conversas** (como um garçom que nunca esquece seu pedido)
4. **Usar ferramentas externas** (como dar superpoderes para a IA)
5. **Processar documentos** (como ter um assistente que lê tudo pra você)

### **ChatGPT vs LangChain - A Diferença na Prática**

**ChatGPT**: É como ter um amigo super inteligente, mas que só pode conversar. Ele não pode:
- Acessar internet
- Executar código
- Lembrar de conversas antigas
- Conectar com outros sistemas

**LangChain**: É como ter um **exército de amigos inteligentes** cada um com uma especialidade, e você é o chefe que coordena tudo!

---

**💡 Dica do Pedro**: LangChain é especialmente útil quando você quer fazer algo mais complexo que uma simples conversa. É tipo a diferença entre pedir um Uber e ter um motorista particular que também é seu assistente pessoal.

<img src='https://cdn.hashnode.com/res/hashnode/image/upload/v1681565061148/fe236ec9-c9cb-4325-8af4-b3b193faadb3.png' width='1200'>


## **Aula 1.2: Setup do Ambiente - Preparando o Terreno**

### **Instalação e Configuração (Sem Complicação)**

Antes de começar a construir, precisamos preparar o terreno. É como aprender a dirigir - primeiro você liga o carro, depois aprende a andar.

Vamos instalar tudo que precisamos:

In [ ]:
# Instalando as dependências necessárias
# Execute esta célula primeiro!

!pip install --quiet langchain openai python-dotenv
!pip install --quiet langchain-community langchain-core
!pip install --quiet --upgrade langchain-huggingface transformers sentencepiece torch
!pip install --quiet huggingface_hub
!pip install --quiet langchain-openai openai

# Para document loaders
!pip install --quiet pypdf python-docx beautifulsoup4 requests youtube-transcript-api

# Para vector stores
!pip install --quiet chromadb faiss-cpu

# Para agents e ferramentas
!pip install --quiet wikipedia duckduckgo-search

# Para deploy e interfaces
!pip install --quiet streamlit gradio fastapi uvicorn

# Para processamento de dados
!pip install --quiet pandas numpy matplotlib seaborn

print("✅ Todas as dependências instaladas com sucesso!")
print("🚀 Agora vamos configurar o LLM...")
print("✅ Dependências instaladas com sucesso!")
print("🚀 Agora vamos importar o que precisamos...")

In [ ]:
# Importando as bibliotecas principais
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Carregando variáveis de ambiente (vamos configurar isso depois)
load_dotenv()

print("📦 Bibliotecas importadas com sucesso!")
print("🔧 Próximo passo: configurar a API key")

### **Configurando a API Key - O "Cartão de Crédito" da IA**

Para usar LangChain com modelos da OpenAI (como GPT), você precisa de uma API key. É como ter um cartão de crédito para pagar pelos serviços de IA.

**💡 Dica importante**: Nunca coloque sua API key diretamente no código! Sempre use variáveis de ambiente.

Vamos criar um arquivo `.env` para guardar suas chaves de forma segura:

In [ ]:
# Criando arquivo .env (se não existir)
if not os.path.exists('.env'):
    with open('.env', 'w') as f:
        f.write('# Suas chaves de API aqui\n')
        f.write('GOOGLE_API_KEY="XXXXCHAVEAQUIXXXXX"\n')
        f.write('OPENAI_API_KEY="XXXXCHAVEAQUIXXXXX"\n')
    print("📝 Arquivo .env criado!")
    print("�� Agora adicione sua API key no arquivo .env")
else:
    print("✅ Arquivo .env já existe!")

# Verificando se a API key está configurada
google_api_key = os.getenv('GOOGLE_API_KEY')
if google_api_key and google_api_key != 'XXXXCHAVEAQUIXXXXX':
    print("🎉 Google API key configurada com sucesso!")
else:
    print("⚠️ Configure sua Google API key no arquivo .env")

# Carregando variáveis do .env
load_dotenv()

def get_llm_colab():
    """Retorna o melhor LLM disponível no Colab"""

    # --- Tentativa 1: Google Gemini (Primeira opção) ---
    try:
        import google.generativeai as genai

        # Configurar API key do Google
        google_key = os.getenv('GOOGLE_API_KEY')
        if google_key:
            genai.configure(api_key=google_key)

            # Criar modelo Gemini
            model = genai.GenerativeModel('gemini-2.0-flash')

            # Criar wrapper para LangChain
            from langchain_google_genai import ChatGoogleGenerativeAI
            llm_google = ChatGoogleGenerativeAI(
                model="gemini-2.0-flash",
                temperature=0.7,
                google_api_key=google_key
            )
            print("✅ Google Gemini configurado!")
            return llm_google

    except Exception as e:
        print(f"⚠️ Google Gemini falhou: {e}")

    # --- Tentativa 2: Hugging Face Local ---
    try:
        # Modelo local
        tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-2.7B")
        model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-2.7B")
        text_generator = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            device=-1,
            max_new_tokens=150,
            do_sample=True,
            temperature=0.9,
            top_p=0.95,
            repetition_penalty=1.5,
            pad_token_id=tokenizer.eos_token_id
        )

        llm_hf = HuggingFacePipeline(pipeline=text_generator)
        print("✅ Hugging Face (local) configurado!")
        return llm_hf

    except Exception as e:
        print(f"⚠️ Hugging Face falhou: {e}")

    # --- Tentativa 3: OpenAI ---
    try:
        from langchain_openai import ChatOpenAI
        openai_key = os.getenv("OPENAI_API_KEY")
        if openai_key:
            llm_openai = ChatOpenAI(
                model="gpt-3.5-turbo",
                temperature=0.7,
                api_key=openai_key
            )
            print("✅ OpenAI configurado!")
            return llm_openai

    except Exception as e:
        print(f"⚠️ OpenAI falhou: {e}")

    # --- Opção 4: Mock LLM ---
    from langchain.llms.fake import FakeListLLM
    respostas = [
        "Aqui está a resposta:", "Vou te ajudar:", "Essa é uma boa pergunta!",
        "Deixe-me explicar:", "Aqui estão as informações:", "Vou criar um exemplo:"
    ]
    print("ℹ️ Usando FakeListLLM como fallback")
    return FakeListLLM(responses=respostas)

# --- Configurando o LLM ---
llm = get_llm_colab()

print("�� LLM configurado no Colab!")
print(f"🤖 Tipo: {type(llm).__name__}")
print("💡 Agora você pode usar 'llm' em todos os exemplos do curso!")

### **Primeiro "Hello World" com LangChain**

Agora vamos fazer nosso primeiro teste! É como dar a primeira volta no carro - simples, mas emocionante.

Vamos criar um modelo básico e fazer uma pergunta simples:

In [ ]:
# Fazendo nossa primeira pergunta!
# É como fazer a primeira pergunta para um amigo novo

try:
    message = HumanMessage(content="O que é LangChain?")
    response = llm.invoke([message])

    print("💬 Chat com DeepSeek:")
    print("=" * 50)
    print(f" Pergunta: {message.content}")
    print(f"🤖 Resposta: {response}")
    print("=" * 50)

except Exception as e:
    print(f"❌ Erro: {e}")

### **Parou aqui e entendeu!** 🎯

Se você conseguiu ver a resposta da IA, **parabéns!** Você acabou de fazer seu primeiro app com LangChain!

**O que acabamos de fazer:**
1. ✅ Instalamos o LangChain
2. ✅ Configuramos a API key
3. ✅ Criamos um modelo
4. ✅ Fizemos uma pergunta e recebemos resposta

**Por que isso é diferente do ChatGPT normal?**
- No ChatGPT, você só conversa
- Com LangChain, você **programa** a conversa
- Você pode automatizar, integrar, personalizar
- É como a diferença entre **usar** um app e **criar** um app

---

### **Teste Rápido - Vamos Experimentar Mais!**

Agora vamos fazer um teste mais interessante. Vamos criar um "assistente de programação" que fala como o Pedro Guth:

In [ ]:
# Criando um assistente com personalidade
# É como configurar um amigo com características específicas

try:
    # Definindo a personalidade do assistente (System Message)
    system_message = SystemMessage(content="""
    Você é o Pedro Guth, um instrutor de programação descontraído e direto.
    Use linguagem informal, faça piadas leves e use analogias do dia a dia.
    Explique conceitos técnicos de forma simples e engraçada.
    Use palavrões com moderação e função pedagógica.
    """)

    # Pergunta do usuário
    user_message = HumanMessage(content="""
    Explique o que é uma variável em programação de forma descontraída,
    como se fosse o Pedro Guth explicando.
    """)

    # Enviando as duas mensagens (sistema + usuário)
    response = llm.invoke([system_message, user_message])

    print("🎭 Assistente com Personalidade:")
    print("=" * 50)
    print(f"🤖 Pedro Guth: {response.content}")
    print("=" * 50)

except Exception as e:
    print(f"❌ Erro: {e}")

### **Na Prática, Meu Consagrado!** 💪

**O que você acabou de ver:**

1. **System Message**: É como dar instruções para um ator antes da peça. "Você vai interpretar o Pedro Guth"
2. **Human Message**: É o que o usuário pergunta
3. **Response**: É a resposta personalizada

**Por que isso é poderoso:**
- Você pode criar **diferentes personalidades** para diferentes usos
- Um assistente para crianças, outro para CEOs, outro para programadores
- É como ter **múltiplos funcionários especializados** em uma IA só

---

### **Comparação: Com vs Sem LangChain**

**Sem LangChain (código manual):**
```python
# Você teria que fazer isso tudo manualmente:
import requests
import json

headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/json'
}

data = {
    'model': 'gpt-3.5-turbo',
    'messages': [
        {'role': 'system', 'content': 'Seja o Pedro Guth...'},
        {'role': 'user', 'content': 'Explique variáveis...'}
    ],
    'temperature': 0.7
}

response = requests.post('https://api.openai.com/v1/chat/completions',
                        headers=headers, json=data)
result = response.json()
answer = result['choices'][0]['message']['content']
```

**Com LangChain:**
```python
# Tudo isso em 3 linhas:
llm = ChatOpenAI(model="gpt-3.5-turbo")
response = llm.invoke([system_message, user_message])
print(response.content)
```

**Diferença**: 20 linhas vs 3 linhas. É como a diferença entre **andar a pé** e **pegar um Uber**! 🚗

---

### **Resumo do que Aprendemos** 📚

✅ **O que é LangChain**: Framework que simplifica o desenvolvimento com IA
✅ **Por que usar**: Economiza tempo, código e dor de cabeça
✅ **Setup básico**: Instalação, configuração de API key
✅ **Primeiro app**: Conversa simples com IA
✅ **Personalização**: Como criar assistentes com personalidade

<img src='https://miro.medium.com/v2/resize:fit:1400/1*8a-WY00_dm43wchuMD9fJg.png' width='1200'>

**🎯 Próximo módulo**: Vamos aprender sobre **Prompts** - a arte de falar com IA de forma eficiente!

---

**💡 Desafio para casa**: Tente criar um assistente que fale como um chef de cozinha e explique como fazer um bolo de chocolate. Use a mesma técnica que aprendemos!